In [7]:
import pyaudio
import numpy as np
import librosa
from pythonosc import udp_client

# PyAudio configuration
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100
CHUNK = 2048

# OSC Configuration
OSC_IP = "127.0.0.1"
OSC_PORT = 6448
OSC_ADDRESS = "/wek/inputs"

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Initialize OSC client
client = udp_client.SimpleUDPClient(OSC_IP, OSC_PORT)

def frequency_to_note(frequency):
    # Mapping frequency to the nearest musical note (chromatically)
    if frequency is not None and frequency > 0:
        midi_num = librosa.hz_to_midi(frequency)  # Convert frequency to MIDI number
        note = librosa.midi_to_note(int(midi_num))  # Convert MIDI number to note
        return note
    return None




def process_audio(in_data, frame_count, time_info, status):
    audio_data = np.frombuffer(in_data, dtype=np.float32)

    # Calculate volume (RMS of the signal)
    volume = np.sqrt(np.mean(np.square(audio_data)))

    # Normalize the volume to a 0-1 range and then scale to 0-127
    normalized_volume = np.clip(volume / np.max(audio_data), 0, 1)
    scaled_volume = int(normalized_volume * 127)

    # Perform FFT and find the dominant frequency
    fft = np.abs(np.fft.fft(audio_data))
    freqs = np.fft.fftfreq(len(fft), 1.0/RATE)
    idx = np.argmax(fft)
    freq = freqs[idx]

    # Convert frequency to musical note
    note = frequency_to_note(freq)

    if note:
        # Ensure the note is a string and the volume is an integer within MIDI range
        note = str(note)
        volume = scaled_volume

        # Print the values before sending (for debugging)
        print(f"Preparing to send Note: {note}, Volume: {volume}")

        try:
            # Send note and volume as an OSC message
            client.send_message(OSC_ADDRESS, [note, volume])
            print(f"Sent Note: {note}, Volume: {volume}")
        except ValueError as e:
            print(f"Error sending OSC message: {e}")

    return (in_data, pyaudio.paContinue)

# Open stream
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK,
                    stream_callback=process_audio)

# Start the stream
stream.start_stream()

# Keep the stream open
try:
    while stream.is_active():
        pass
except KeyboardInterrupt:
    # Stop and close the stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    audio.terminate()
    print("Stream stopped")

Preparing to send Note: B2, Volume: 22
Sent Note: B2, Volume: 22
Preparing to send Note: E0, Volume: 47
Sent Note: E0, Volume: 47
Preparing to send Note: E0, Volume: 61
Sent Note: E0, Volume: 61
Preparing to send Note: E0, Volume: 44
Sent Note: E0, Volume: 44
Preparing to send Note: G♯2, Volume: 60
Sent Note: G♯2, Volume: 60
Preparing to send Note: E0, Volume: 72
Sent Note: E0, Volume: 72
Preparing to send Note: E0, Volume: 59
Sent Note: E0, Volume: 59
Preparing to send Note: E0, Volume: 66
Sent Note: E0, Volume: 66
Preparing to send Note: E0, Volume: 49
Sent Note: E0, Volume: 49
Preparing to send Note: B1, Volume: 50
Sent Note: B1, Volume: 50
Preparing to send Note: E2, Volume: 61
Sent Note: E2, Volume: 61
Preparing to send Note: F♯3, Volume: 52
Sent Note: F♯3, Volume: 52
Preparing to send Note: E1, Volume: 54
Sent Note: E1, Volume: 54
Preparing to send Note: E1, Volume: 57
Sent Note: E1, Volume: 57
Preparing to send Note: B1, Volume: 45
Sent Note: B1, Volume: 45
Preparing to send Not